# scRNA cell population change test

The purpose of this method is to test whether cell populations sizes significantly different through $M$ covariats $X^{N\times M}$. We assume the following data generating process of cell population counts $y_{ik}$ of $i\in\{1,..,N\}$ samples and $k\in\{1,..,K\}$ detected cell populations:

\begin{align}
y_i &\sim\text{DirMult}(y_i^+,\gamma_i)\\
y_i^+ &= \sum_{k=1}^K y_{i,k}\\
\log(\gamma_{ij}) &\sim \alpha_j + x_i^T\beta_j + V_{ij} \\
\alpha &\sim \text{MvN}(\mu_0, \sigma_{0}^2 I)\\
\beta_{j,k} &\sim \text{N}(0, \nu_{\beta,j}^2\sigma_{\beta,k}^2)\\
V_i &\sim \text{MvN}(\mathbf{0}, \Sigma_i)\\
\sigma_0^2 &\sim \text{HalfCauchy}(0, 5)\\
\sigma_{\beta,k}^2 &\sim \text{HalfCauchy}(0, 5)\\
\nu_{\beta,j}^2 &\sim \text{HalfCauchy}(0, 1)\\
\Sigma_i &\sim \text{Inv-Wishart}(\Psi_0, \rho_0)
\end{align}

$V_i$ characterize the unobserved characteristics that are associated with the mean count for cell type $k$ in subject $i$ and account for within-subject correlations.



**Software requirements**
- Python == 3.6
- Tensorflow (nightly)
- Tensorflow-probability (nightly, >= 0.6.0)

In [1]:
%matplotlib inline

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import edward2 as ed
import warnings

plt.style.use("ggplot")
warnings.filterwarnings('ignore')

tfd = tfp.distributions
tfb = tfp.bijectors
plt.style.use('ggplot')


def make_simple_model(X,k,n_total):
    """
    :param X: The feature matrix
    :param y: cell counts
    :param n_total: vector of total number of  
    :returns: The tensorflow model
    """
    n,m = X.shape
    #hyperprios
    nu = ed.HalfCauchy(0,1, sample_shape=m, name="nu")
    sigma_beta = ed.HalfCauchy(0, 5, sample_shape=k, name="sigma_beta")
    sigma_alpha = ed.HalfCauchy(0, 5, sample_shape=k, name="sigma_alpha")

        
    #priors
    alpha = ed.Normal(tf.zeros(k),
                     scale=sigma_alpha, 
                     name="alpha")
    beta =  ed.Normal(tf.zeros([m,k]),
                      scale=tf.tensordot(nu,sigma_beta, axes=0), 
                      name="beta")

    p = ed.Dirichlet(tf.exp(alpha + tf.matmul(X,beta)), name="p")
    outcomes = ed.Multinomial(total_count=n_total, 
                                probs=p, 
                                name="outcomes")

    return outcomes
    

In [6]:
n=10; m=3; k=5; m_r=1; k_r=1
def generate_data(n=n, m=m, k=k, m_r=m_r, k_r=k_r,rho=0.4):

    # Covaraits drawn from MvNormal(0, Cov) Cov_ij = p ^|i-j| , p=0.4
    #Tibshirani for correlated covariats Tibshirani (1996) 
    #X_cov =np.array([[rho**np.abs(i-j) for j in range(k)] for i in range(k)])
    X_cov = np.eye(m)
    X = np.random.multivariate_normal(np.zeros(m), cov=X_cov, size=n).astype(np.float32)
 
    k_r_idx = np.random.choice(range(k), size=m_r, replace=False)
    m_r_idx = np.random.choice(range(m), size=k_r, replace=False)

    # slope and intercepts
    alphas = np.random.uniform(-2.3, 2.3, size=k)
    betas = np.zeros((m,k))

    for i in m_r_idx:
        for j in k_r_idx:
            #p = np.random.binomial(n=1, p= 0.5)
            betas[i,j] = 2. 
    
    xi = 0.01 #dispersion parameter
    gamma = np.exp(alphas + np.matmul(X, betas))
    #gamma_plus = gamma_raw.sum(axis=1)
    #gamma = (gamma_raw/gamma_plus[:, None]) *((1-xi)/xi)
    
    y = np.zeros((n, k))
    
    # sample total number of reads:
    n_total = np.random.randint(100, 500, size=n).astype(np.float32)
    n_total =500
    print(n_total)
    for i in range(n):
        pi = np.random.dirichlet(gamma[i,:])
        #y[i,:] = np.random.multinomial(n_total[i], pi).astype(np.float32)
        y[i,:] = np.random.multinomial(n_total, pi).astype(np.float32)
    print()
    return n_total,X,y,m_r_idx, k_r_idx



In [7]:
#sess = tf.Session()
n_data, X_data, y_data, m_r_idx, k_r_ids = generate_data()
#n_data = tf.to_float(n_data)
#X_data = tf.to_float(X_data)
#y_data = tf.to_float(y_data)
print("y",y_data)
#with sess.as_default():
model = make_simple_model(X_data,k,n_data)

500

y [[  7.   9.   0.   7. 477.]
 [ 20.   9.  29.   1. 441.]
 [ 88.   0.  61.   0. 351.]
 [ 37.  55. 234.   0. 174.]
 [116.  61.  77.   0. 246.]
 [  4.   0. 118.   4. 374.]
 [  0.   3. 478.   1.  18.]
 [ 72.  21.   0.   1. 406.]
 [ 41.   0.  19.  36. 404.]
 [ 69.   5.  75.   0. 351.]]


### Inference

In [11]:
#model_template = tf.make_template("model", model)


In [8]:
log_joint = ed.make_log_joint_fn(model)

def target_log_prob_fn(alpha, beta, nu, sigma_alpha, sigma_beta, p):
  """Unnormalized target density as a function of states."""
  return log_joint(
    X_data,
    k,
    n_data,
    alpha=alpha,
    beta=beta,
    nu=nu,
    sigma_alpha=sigma_alpha,
    sigma_beta=sigma_beta,
    p=p,
    outcomes=y_data,
  )


# MCMC setup
qnu = tf.zeros([m], name="qnu")  # initial state
qsigma_alpha = tf.zeros([k], name="qsigma_alpha")  # initial state
qsigma_beta = tf.zeros([k], name="qsigma_beta")  # initial state
qalpha = tf.ones([k], name="qalpha")  # initial state
qbeta = tf.ones([k,m], name="qbeta")  # initial state
qp = tf.ones([k,m], name="qp")  # initial state


states, kernels_results = tfp.mcmc.sample_chain(
    num_results=5000,
    num_burnin_steps=3000,
    current_state=[qalpha, qbeta, qnu, qsigma_alpha, qsigma_beta,qp],
    kernel=tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=target_log_prob_fn,
        step_size=0.01,
        num_leapfrog_steps=5))


nu_, sigma_alpha_, sigma_beta_, alpha_, beta_, p_ = states

with tf.Session() as sess:
  [
      nu_,
      sigma_alpha_,
      sigma_beta_,
      alpha_,
      beta_,
      p_
  ] = sess.run([
      nu_,
      sigma_alpha_,
      sigma_beta_,
      alpha_,
      beta_,
      p_,
      kernel_results.is_accepted,
  ])


num_accepted = np.sum(is_accepted_)
print('Acceptance rate: {}'.format(num_accepted / num_results))

Instructions for updating:
Use tf.random.categorical instead.


TypeError: unsupported callable

In [1]:
    from scipy.special import softmax
    import numpy as np
    import arviz as az

    # Settings
    D = 4  # number of dimensions
    N = 100  # number of datapoints to generate
    K = 5
    n_total = [1000]*N

    noise_std_true = 1.0

    # Generate data
    b_true = np.random.randn(K).astype(np.float32)  # bias (alpha)
    w_true = np.random.randn(D, K).astype(np.float32)  # weights (beta)
    x = np.random.randn(N, D).astype(np.float32)
    noise = noise_std_true * np.random.randn(N, 1).astype(np.float32)
    concentration = softmax(np.matmul(x, w_true) + b_true + noise)
    y = np.zeros([N, K], dtype=np.float32)
    for i in range(N):
        y[i, :] = np.random.multinomial(n_total[i], concentration[i, :])

/home/benjamin/miniconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from scipy.stats import norm

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow_probability import edward2 as ed
tfd = tfp.distributions

sns.set()
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
%config InlineBackend.figure_format = 'svg'
np.random.seed(111)
tf.set_random_seed(111)

def linear_regression(X,K,n_total):
  N,D = X.shape      #number of dimensions
  sigma_alpha = ed.HalfCauchy(tf.zeros([K]), tf.ones([K])*5, name="sigma_alpha")
  sigma_beta = ed.HalfCauchy(tf.zeros([K]), tf.ones([K])*5, name="simga_beta")
  nu = ed.HalfCauchy(tf.zeros([D]), tf.ones([D]), name="nu")
  coeffs = ed.Normal(        #normal prior on weights
      loc=tf.zeros([D,K]),
      scale=tf.ones([D,K]),
      name="beta")
  bias = ed.Normal(          #normal prior on bias
      loc=tf.zeros([K]), 
      scale=tf.ones([K]),
      name="alpha") 
  predictions = ed.DirichletMultinomial(   #normally-distributed noise
      n_total,
      concentration=tf.exp(tf.matmul(X, coeffs)+bias),
      name="predictions")
  return predictions

In [15]:
log_joint = ed.make_log_joint_fn(linear_regression)

In [16]:
# Function to compute the log posterior probability
def target_log_prob_fn(alpha, beta):
  return log_joint(
      X=x,
      predictions=y,
      K=K,
      n_total=n_total,
      beta=beta,
      alpha=alpha
)

In [17]:
# HMC Settings
num_results = int(10e3) #number of hmc iterations
n_burnin = int(5e3)     #number of burn-in steps
step_size = 0.01
num_leapfrog_steps = 10

# Parameter sizes
beta_size = [D,K]
alpha_size = [K]

# HMC transition kernel
kernel = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=target_log_prob_fn,
    step_size=step_size,
    num_leapfrog_steps=num_leapfrog_steps)

# Define the chain states
states, kernel_results = tfp.mcmc.sample_chain(
    num_results=num_results,
    num_burnin_steps=n_burnin,
    kernel=kernel,
    current_state=[
        tf.zeros(alpha_size, name='init_alpha'),
        tf.zeros(beta_size, name='init_beta'),
    ])
alpha, beta = states

with  tf.Session() as sess:
  [
      alpha_,
      beta_,
      is_accepted_,
  ] = sess.run([
      alpha,
      beta,
      kernel_results.is_accepted,
  ])

# Samples after burn-in
alpha_samples = alpha_[n_burnin:,:]
beta_samples = beta_[n_burnin:,:,:]
accepted_samples = is_accepted_[n_burnin:]

In [18]:
print('Acceptance rate: %0.1f%%' % (100*np.mean(accepted_samples)))


Acceptance rate: 98.5%


In [20]:
alpha_samples.mean(axis=0)

array([-2.526279 , -0.7007637, -1.653993 , -2.4317858,  6.3784385],
      dtype=float32)